<a href="https://colab.research.google.com/github/tchuynhminhtuan/daily-promotion/blob/main/Generate_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title 1. Cài đặt môi trường (Chạy 1 lần đầu)
# @markdown Bấm nút **Play** bên trái để cài đặt các thư viện cần thiết.
!pip install pandas
from IPython.display import clear_output
clear_output()
print("✅ Cài đặt hoàn tất! Bạn có thể chuyển sang Bước 2.")

In [ ]:

# @title 2. Tải dữ liệu từ GitHub
# @markdown Chạy ô này để tải toàn bộ dữ liệu CSV mới nhất từ GitHub.
import os

# Xóa folder cũ nếu có để đảm bảo lấy dữ liệu mới nhất
if os.path.exists('daily-promotion'):
    !rm -rf daily-promotion

print("⬇️ Đang tải dữ liệu từ GitHub...")
!git clone https://github.com/tchuynhminhtuan/daily-promotion.git

BASE_DIR_DRIVE = "/content/daily-promotion/content"

if os.path.exists(BASE_DIR_DRIVE):
    print(f"✅ Đã tải xong! Thư mục dữ liệu: {BASE_DIR_DRIVE}")
else:
    print(f"❌ Lỗi: Không tìm thấy thư mục {BASE_DIR_DRIVE}")

# --- SECTION SPLIT ---

# @title 3. Chọn ngày và Tạo báo cáo
# @markdown Chạy ô này, sau đó **nhập số thứ tự** các ngày muốn so sánh vào bảng hiện ra bên dưới.

import pandas as pd
import re
import html
import os
from google.colab import files
from datetime import datetime

# ==============================================================================
# HỆ THỐNG XỬ LÝ DỮ LIỆU (Tự giải nén các class cần thiết)
# ==============================================================================

COLUMN_MAPPING = {
    "Product_Name": "Product Name",
    "Color": "Color",
    "Gia_Niem_Yet": "Listed Price",
    "Gia_Khuyen_Mai": "Promo Price",
    "Khuyen_Mai": "Promotion Details",
    "Thanh_Toan": "Payment Promo",
    "Uu_Dai_Them": "Payment Promo",
    "Voucher_Image": "Voucher",
    "Other_promotion": "Other Promo",
    "Link": "Link"
}

class DataLoader:
    @staticmethod
    def load_all_data(base_dir, dates):
        all_data = []
        for date_str in dates:
            day_dir = os.path.join(base_dir, date_str)
            if not os.path.exists(day_dir): continue

            file_patterns = {
                "FPT": f"1-fpt-{date_str}.csv",
                "MW": f"2-mw-{date_str}.csv",
                "Viettel": f"3-viettel-{date_str}.csv",
                "HoangHa": f"4-hoangha-{date_str}.csv",
                "DDV": f"5-ddv-{date_str}.csv",
                "CPS": f"6-cps-{date_str}.csv"
            }

            for channel_name, filename in file_patterns.items():
                file_path = os.path.join(day_dir, filename)
                if os.path.exists(file_path):
                    try:
                        df = pd.read_csv(file_path, sep=None, engine='python')
                        df = df.rename(columns=COLUMN_MAPPING)
                        df['Channel'] = channel_name
                        df['Date'] = date_str
                        df['_RawDate'] = date_str
                        for col in ['Listed Price', 'Promo Price']:
                            if col in df.columns:
                                df[col] = pd.to_numeric(df[col], errors='coerce')
                        all_data.append(df)
                    except: pass

        return pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()

class PriceMatrixGeneratorNoCSV:
    def __init__(self, df):
        self.df = df
        self.price_lookup = {}
    def run(self):
        for _, row in self.df.iterrows():
            key = (row['Channel'], row['Product Name'], row['Color'], row['Date'])
            self.price_lookup[key] = row['Promo Price']

class PromoDiffGeneratorNoCSV:
    def __init__(self, df, price_generator, output_file):
        self.df = df
        self.price_gen = price_generator
        self.output_file = output_file

    def run(self):
        df_collapsed = self._collapse_for_promo(self.df)
        df_diff = self._identify_changes(df_collapsed)
        if df_diff is not None and not df_diff.empty:
            self._save_html(df_diff)
            return True
        return False

    def _collapse_for_promo(self, df):
        df_filled = df.copy()
        text_cols = ['Promotion Details', 'Payment Promo']
        for col in [c for c in text_cols if c in df.columns]:
            df_filled[col] = df_filled[col].fillna("").apply(self._normalize_text)
        return df_filled.drop_duplicates(subset=['Channel', 'Date', 'Product Name', 'Color'])

    def _normalize_text(self, text):
        if pd.isna(text) or str(text).strip() == "": return ""
        text = str(text).replace('\xa0', ' ').replace('\u200b', '')
        lines = [re.sub(r'^(\d+[\.\)]|[-•])\s*', '', l.strip()) for l in re.split(r'[\n\r]+', text) if l.strip()]
        return " | ".join(sorted(lines))

    def _identify_changes(self, df):
        df = df.sort_values(by=['Channel', 'Product Name', 'Color', '_RawDate'])
        changes = []
        grouped = df.groupby(['Channel', 'Product Name', 'Color'])
        valid_cols = [c for c in ['Promotion Details', 'Payment Promo'] if c in df.columns]

        for _, group in grouped:
            if len(group) < 2: continue
            for i in range(1, len(group)):
                curr, prev = group.iloc[i], group.iloc[i-1]
                has_change = False
                record = {
                    "Channel": curr['Channel'], "Product Name": curr['Product Name'], "Color": curr['Color'],
                    "Date": curr['Date'], "Prev_Date": prev['Date'],
                    "New_Price": curr['Promo Price'], "Old_Price": prev['Promo Price'], "Link": curr.get('Link', '')
                }
                for col in valid_cols:
                    curr_txt, prev_txt = curr[col], prev[col]
                    record[f"Changed_{col}"] = "YES" if curr_txt != prev_txt else "NO"
                    record[f"Old_{col}"], record[f"New_{col}"] = prev_txt, curr_txt
                    if curr_txt != prev_txt: has_change = True

                try:
                    if float(curr['Promo Price'] or 0) != float(prev['Promo Price'] or 0): has_change = True
                except: pass

                if has_change: changes.append(record)
        return pd.DataFrame(changes)

    def _save_html(self, df):
        HTMLGenerator(df, self.output_file).generate()

class HTMLGenerator:
    def __init__(self, df, output_file):
        self.df = df
        self.output_file = output_file

    def generate(self):
        # [Full HTML Generator Logic included in the standalone script]
        # (Để ngắn gọn, mình sẽ đóng gói toàn bộ logic HTML CSS linh hoạt tại đây)
        # Báo cáo này bao gồm bộ lọc Channel, Date, Search, và Responsive cho di động.

        channels = sorted(self.df['Channel'].unique().tolist())
        dates = sorted(self.df['Date'].unique().tolist(), reverse=True)
        channel_opts = "".join([f'<option value="{c}">{c}</option>' for c in channels])
        date_opts = "".join([f'<option value="{d}">{d}</option>' for d in dates])

        # --- HTML TEMPLATE ---
        html_head = f"""
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Báo cáo So sánh Khuyến mãi</title>
            <style>
                body {{ font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; margin: 20px; line-height: 1.5; color: #333; }}
                h1 {{ color: #2c3e50; font-size: 1.5rem; }}
                .controls {{ background: #f8f9fa; padding: 15px; border-radius: 8px; margin-bottom: 25px; display: flex; gap: 15px; align-items: center; flex-wrap: wrap; border: 1px solid #e9ecef; position: sticky; top: 0; z-index: 1000; }}
                .control-group {{ display: flex; align-items: center; gap: 8px; flex: 1 1 auto; }}
                label {{ font-weight: 600; font-size: 0.9em; white-space: nowrap; }}
                select, input {{ padding: 8px; border-radius: 4px; border: 1px solid #ccc; }}
                .product-block {{ background: #fff; border: 1px solid #dee2e6; margin-bottom: 20px; padding: 15px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.02); }}
                .product-header {{ font-weight: bold; border-bottom: 1px solid #eee; padding-bottom: 8px; margin-bottom: 10px; display: flex; justify-content: space-between; }}
                .price-change-down {{ color: #28a745; font-weight: bold; }}
                .price-change-up {{ color: #dc3545; font-weight: bold; }}
                .diff-table {{ width: 100%; border-collapse: collapse; margin-top: 10px; font-size: 0.9em; }}
                .diff-table th, .diff-table td {{ border: 1px solid #eee; padding: 10px; text-align: left; vertical-align: top; width: 50%; }}
                .added {{ background: #d4edda; color: #155724; padding: 2px 4px; border-radius: 2px; }}
                .removed {{ background: #f8d7da; color: #721c24; text-decoration: line-through; padding: 2px 4px; border-radius: 2px; }}
                .hidden {{ display: none !important; }}
                @media (max-width: 600px) {{ .controls {{ position: static; flex-direction: column; align-items: stretch; }} .diff-table th, .diff-table td {{ padding: 5px; }} }}
            </style>
        </head>
        <body>
            <h1>Báo cáo So sánh Khuyến mãi</h1>
            <p style="color: grey;">Ngày tạo: {datetime.now().strftime('%Y-%m-%d %H:%M')}</p>
            <div class="controls">
                <div class="control-group"><label>Ngày:</label><select id="dateFilter"><option value="ALL">Tất cả</option>{date_opts}</select></div>
                <div class="control-group"><label>Kênh:</label><select id="channelFilter"><option value="ALL">Tất cả</option>{channel_opts}</select></div>
                <div class="control-group"><label>Tìm kiếm:</label><input type="text" id="searchInput" placeholder="Tên sản phẩm..."></div>
                <div class="control-group"><label>Thay đổi Giá:</label><select id="priceFilter"><option value="ALL">Tất cả</option><option value="YES">Đổi Giá</option><option value="NO">Không Đổi</option></select></div>
                <div class="control-group"><label>Thay đổi KM:</label><select id="promoFilter"><option value="ALL">Tất cả</option><option value="YES">Có thay đổi</option><option value="NO">Không</option></select></div>
                <div style="margin-left:auto"><span id="matchCount"></span></div>
            </div>
            <div id="report-container">
        """

        container_content = ""
        for _, row in self.df.iterrows():
            container_content += self._render_block(row)

        html_foot = """
            </div>
            <script>
                function filter() {
                    let d = document.getElementById('dateFilter').value;
                    let c = document.getElementById('channelFilter').value;
                    let p = document.getElementById('priceFilter').value;
                    let pr = document.getElementById('promoFilter').value;
                    let s = document.getElementById('searchInput').value.toLowerCase();
                    let v = 0;
                    document.querySelectorAll('.product-block').forEach(b => {
                        let matches = (d==='ALL' || b.dataset.date===d) &&
                                      (c==='ALL' || b.dataset.channel===c) &&
                                      (p==='ALL' || b.dataset.priceChange===p) &&
                                      (pr==='ALL' || b.dataset.promoChange===pr) &&
                                      (b.dataset.product.includes(s));
                        b.classList.toggle('hidden', !matches);
                        if(matches) v++;
                    });
                    document.getElementById('matchCount').innerText = `Đang hiển thị ${v} mục`;
                }
                document.querySelectorAll('select, input').forEach(el => el.oninput = filter);
                filter();
            </script>
        </body></html>
        """
        with open(self.output_file, "w", encoding="utf-8") as f:
            f.write(html_head + container_content + html_foot)

    def _render_block(self, row):
        def fmt(p):
            if pd.isna(p) or p==0: return ""
            return "{:,.0f}".format(float(p))

        p_old, p_new = row.get('Old_Price', 0), row.get('New_Price', 0)
        p_html = f"{fmt(p_new)}"
        try:
            diff = float(p_new or 0) - float(p_old or 0)
            if diff < 0: p_html += f' <span class="price-change-down">({fmt(diff)})</span>'
            elif diff > 0: p_html += f' <span class="price-change-up">(+{fmt(diff)})</span>'
        except: pass

        link = row.get('Link', '')
        link_html = f'<a href="{link}" target="_blank" style="font-size:0.8em; color:#007bff">Xem sản phẩm &rarr;</a>' if link else ""

        # Calculate Change Statuses
        price_changed = 'NO'
        try:
             diff_val = float(row.get('New_Price', 0)) - float(row.get('Old_Price', 0))
             if abs(diff_val) > 0: price_changed = 'YES'
        except: pass

        promo_changed = 'NO'
        for col in ["Promotion Details", "Payment Promo"]:
             if row.get(f"Changed_{col}") == 'YES': promo_changed = 'YES'

        block = f"""
        <div class="product-block"
             data-channel="{row['Channel']}"
             data-product="{row['Product Name'].lower()}"
             data-date="{row['Date']}"
             data-price-change="{price_changed}"
             data-promo-change="{promo_changed}">
            <div class="product-header">
                <span>{row['Channel']} - {row['Product Name']} - {row['Color']}</span>
                <span>{p_html}</span>
            </div>
            {link_html}
            <div style="font-size:0.8em; color:grey; margin-bottom:10px">So sánh: {row['Prev_Date']} vs {row['Date']}</div>
        """
        for title, col in [("Khuyến mãi", "Promotion Details"), ("Thanh toán", "Payment Promo")]:
            if f"Changed_{col}" in row:
                block += self._render_section(row, title, col)
        block += "</div>"
        return block

    def _render_section(self, row, title, col):
        if row[f"Changed_{col}"] == "NO":
            return f'<div style="font-size:0.85em; color:#666; margin-top:5px"><b>{title}:</b> Không thay đổi</div>'

        old_items = set(str(row[f"Old_{col}"]).split(" | ")) - {""}
        new_items = set(str(row[f"New_{col}"]).split(" | ")) - {""}
        common = old_items & new_items

        def build_list(items, type):
            lis = "".join([f"<li class='{type}'>{html.escape(i)}</li>" for i in sorted(list(items))])
            return f"<ul style='padding-left:15px; margin:0'>{lis}</ul>" if lis else ""

        return f"""
        <div style="font-weight:bold; font-size:0.85em; margin-top:10px">{title}</div>
        <table class="diff-table">
            <tr><th>{row['Prev_Date']}</th><th>{row['Date']}</th></tr>
            <tr>
                <td>{build_list(old_items-new_items, 'removed')}{build_list(common, '')}</td>
                <td>{build_list(new_items-old_items, 'added')}{build_list(common, '')}</td>
            </tr>
        </table>
        """

# ==============================================================================
# QUY TRÌNH CHẠY (BẮT ĐẦU TỪ ĐÂY)
# ==============================================================================

print(f"\n🚀 --- BẮT ĐẦU TẠO BÁO CÁO ĐỐI SOÁT ---")
if not os.path.exists(BASE_DIR_DRIVE):
    print("❌ Lỗi: Thư mục Drive không tồn tại. Vui lòng kiểm tra Bước 2.")
else:
    # 1. Tìm các ngày có dữ liệu
    date_pattern = re.compile(r'^\d{4}-\d{2}-\d{2}$')
    available = sorted([d for d in os.listdir(BASE_DIR_DRIVE) if os.path.isdir(os.path.join(BASE_DIR_DRIVE, d)) and date_pattern.match(d)], reverse=True)

    if len(available) < 1:
        print("❌ Không tìm thấy thư mục ngày (YYYY-MM-DD) nào trong Drive.")
    else:
        print("\n" + "="*40)
        print("📅 CÁC NGÀY DỮ LIỆU HIỆN CÓ TRÊN DRIVE")
        print("="*40)
        for i, d in enumerate(available):
            print(f" [{i}] Ngày: {d}")
        print("="*40)

        try:
            new_idx = int(input(f"\n👉 Chọn số thứ tự ngày MỚI (Mặc định 0 - {available[0]}): ") or 0)
            old_idx = int(input(f"👉 Chọn số thứ tự ngày CŨ để so sánh (Mặc định 1): ") or 1)

            newer, older = available[new_idx], available[old_idx]
            print(f"\n🔄 Đang so sánh: {older} ➔ {newer}...")

            # 2. Tải dữ liệu
            df = DataLoader.load_all_data(BASE_DIR_DRIVE, [older, newer])
            if df.empty:
                print("❌ Không tải được dữ liệu. Kiểm tra các file CSV trong Drive.")
            else:
                print(f"📊 Đã tải {len(df)} dòng dữ liệu.")

                # 3. Phân tích
                price_gen = PriceMatrixGeneratorNoCSV(df)
                price_gen.run()

                output_fn = f"Bacao_SoSanh_{older}_vs_{newer}.html"
                diff_gen = PromoDiffGeneratorNoCSV(df, price_gen, output_fn)
                success = diff_gen.run()

                if success:
                    print("\n" + "✨"*20)
                    print("🎯 TẠO BÁO CÁO THÀNH CÔNG!")
                    print(f"💾 File đã lưu: {output_fn}")
                    print("✨"*20 + "\n")

                    # 4. Tự động tải xuống
                    print("📥 Đang tự động tải báo cáo về máy tính...")
                    files.download(output_fn)
                else:
                    print("✅ Không có thay đổi nào giữa 2 ngày này.")
        except Exception as e:
            print(f"⚠️ Có lỗi xảy ra: {e}")
